**This program will parse your data of midi files into data readable by the ai**

Import

In [ ]:
!pip install py_midicsv
!pip install mido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 2.0 MB/s eta 0:00:00


In [ ]:
!rm -rf /content/music/

**Upload your zip file of midi files (all in root of zip) and rename to music.zip also create a folder under the same name in the content folder**

In [ ]:
!unzip /content/music.zip -d /content/music/

Streaming output truncated to the last 5000 lines.
  inflating: /content/music/MIDI/Inawhile.mid  
  inflating: /content/music/MIDI/Incantations4OdeCynthia.mid  
  inflating: /content/music/MIDI/indaclub.mid  
  inflating: /content/music/MIDI/infantlo.mid  
  inflating: /content/music/MIDI/Infected (5).mid  
  inflating: /content/music/MIDI/Innamorare tanto.mid  
  inflating: /content/music/MIDI/Intro (9).mid  
  inflating: /content/music/MIDI/intrust.mid  
  inflating: /content/music/MIDI/in_the_good_old_summertime1-Db160.mid  
  inflating: /content/music/MIDI/in_the_summertime_1.mid  
  inflating: /content/music/MIDI/io ti aspetterò.mid  
  inflating: /content/music/MIDI/iran2.mid  
  inflating: /content/music/MIDI/isntlifestrange.mid  
  inflating: /content/music/MIDI/It Came Upon A Midnight Clear (Xmas) (Seq Harry Todd) midnight_ht.mid  
  inflating: /content/music/MIDI/IT'S.MID  
  inflating: /content/music/MIDI/itsalltoomuch L.mid  
  inflating: /content/music/MIDI/i_hear_you_kno

**Convert the midis into all one track**

In [ ]:
#@title # **Convert the dataset and make sure to change midi dir if you changed it above** *Also* you may get errors but its probably ok
import py_midicsv as pm
import os
import re


midi_dir = "/content/MIDI/" #@param {type:"string"}
midi_files = [f for f in os.listdir(midi_dir) if f.endswith('.mid')]
csv_string = []

def get_note_event(start, note, instrument, length, velocity):
  out_instrument = "%"
  match instrument:
    case 2:
      out_instrument = "%"
    case 3:
      out_instrument = "^"
    case 4:
      out_instrument = "&"
    case 5:
      out_instrument = "*"
    case 6:
      out_instrument = "("
    case _:
      out_instrument = ")"


  dynamic = "$"
  if(velocity > 108):
    dynamic = "$"
  elif velocity > 74:
    dynamic = "#"
  elif velocity > 40:
    dynamic = "@"
  else:
    dynamic = "!"

  return [start, dynamic, length, out_instrument, note]

def parse_split_csv_item(item):
  try:
    item[2] = item[2].replace(' ', '')
    item[0] = int(item[0])
    item[1] = int(item[1])
    item[3] = int(item[3])
    item[4] = int(item[4])
    item[5] = int(item[5])
  except:
    pass;
  return item

# Load the MIDI file and parse it into CSV format
numNotesInCsv = 0
num_filesSoFar = 0
for midi_file in midi_files:
  csv_parsed = []
  try:
    lastNoteStart = 0
    csv_unparsed = pm.midi_to_csv(midi_dir + midi_file)
    notesPlaying = []
    for csv_item in csv_unparsed:
      split_csv_item = csv_item.split(',')
      split_csv_item = parse_split_csv_item(split_csv_item);
      match(split_csv_item[2]):
        case "Note_on_c":
          toAdd = True
          for i in notesPlaying:
            if(i[4] == split_csv_item[4]):
              toAdd = False
          if toAdd:
            notesPlaying.append(split_csv_item)
        case "Note_off_c":
          # Find note that turning off
          i = 0
          while i < len(notesPlaying):
            if(notesPlaying[i][4] == split_csv_item[4]):
              temp = get_note_event(notesPlaying[i][1], split_csv_item[4], split_csv_item[0], split_csv_item[1] - notesPlaying[i][1], notesPlaying[i][5])
              csv_parsed.append(temp)
              #csv_parsed.append(get_note_event(notesPlaying[i][1] - lastNoteStart, split_csv_item[4], 0, split_csv_item[1] - notesPlaying[i][1], notesPlaying[i][5]) + '|')
              #csv_string += get_note_event(notesPlaying[i][1] - lastNoteStart, split_csv_item[4], 0, split_csv_item[1] - notesPlaying[i][1], notesPlaying[i][5]) + '|'
              lastNoteStart = notesPlaying[i][1]
              notesPlaying.pop(i)
              i-=1
            i+=1
  except Exception as e:
    print(e)

  # Change to be delta time instead of abs and sort time
  csv_parsed = sorted(csv_parsed, key=lambda x: x[0])
  for i in range(len(csv_parsed) - 1):
    csv_item_copy = csv_parsed[i].copy()
    if(i != 0):
      csv_item_copy[0] = csv_parsed[i][0] - csv_parsed[i - 1][0]

    # Also concat to csv_parsed
    for j in csv_item_copy:
      csv_string += str(j)
    csv_string += "|"
    numNotesInCsv+=1
    if(numNotesInCsv % 400 == 0):
      csv_string += "\n"
  csv_string += "\n"
  num_filesSoFar+=1
  if(num_filesSoFar == 15000):
    with open("/content/drive/MyDrive/convertedData15.txt", "w") as f:
      f.writelines(csv_string)

#for i in range(0, len(csv_string), 1800):
#  csv_string[i] = "\n"

with open("convertedData.txt", "w") as f:
    f.writelines(csv_string)

Event length mismatch for PortEvent at position 319
Event length mismatch for PortEvent at position 79
Event length mismatch for PortEvent at position 267
Event length mismatch for PortEvent at position 312
Program value is out of range: 154 at position 8048
Event length mismatch for PortEvent at position 86


Event length mismatch for PortEvent at position 27
Event length mismatch for PortEvent at position 45
Event length mismatch for PortEvent at position 151
Event length mismatch for PortEvent at position 60
Event length mismatch for PortEvent at position 74
Event length mismatch for PortEvent at position 98
